# About the Data

* The data was taken from 
the csv file complaints.csv


* The complaints are for the products:<br>

  1. 'Bank account or service'
  2. 'Checking or savings account'
  3. 'Consumer Loan'
  4. 'Credit card or prepaid card'
  5. 'Credit reporting, credit repair services, or other personal consumer reports' 
  6. 'Debt collection'
  7. Money transfer/s, virtual currency, or money service'
  8. 'Mortgage'
  9. 'Payday loan, title loan, or personal loan'
  10. 'Student loan'
  11. 'Vehicle loan or lease'


* The data cleaning was done using spacy library



## Next Steps

* Fine-tune DistilBERT on the sample data 

##Google Drive access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# write the appropriate paths to retrieve the data and store results 
sample_data_path = '/content/drive/MyDrive/Complaints_csv/Experiment3/SAMPLE_25_APRIL_2022_Experiment3.csv'

# Loading the sample dataset

In [ ]:
#Load the data
import pandas as pd
sample_df = pd.read_csv(sample_data_path)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79993 entries, 0 to 79992
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   consumer_complaint_narrative   79993 non-null  object 
 1   product                        79993 non-null  object 
 2   split_words_whitespaces        79993 non-null  object 
 3   number_of_words                79993 non-null  int64  
 4   number_of_charachters          79993 non-null  int64  
 5   charachters_by_words           79993 non-null  int64  
 6   number_of_unique_words         79993 non-null  int64  
 7   potenial_mask_words            79993 non-null  object 
 8   number_of_potenial_mask_words  79993 non-null  int64  
 9   potenial_mask_words_BY_words   79993 non-null  float64
dtypes: float64(1), int64(5), object(4)
memory usage: 6.1+ MB


In [ ]:
# drop the duplicate entries which are not maximum length
sample_df = sample_df.loc[sample_df['number_of_words'].groupby(sample_df['consumer_complaint_narrative']).idxmax()].sort_values('product')
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77895 entries, 1039 to 79231
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   consumer_complaint_narrative   77895 non-null  object 
 1   product                        77895 non-null  object 
 2   split_words_whitespaces        77895 non-null  object 
 3   number_of_words                77895 non-null  int64  
 4   number_of_charachters          77895 non-null  int64  
 5   charachters_by_words           77895 non-null  int64  
 6   number_of_unique_words         77895 non-null  int64  
 7   potenial_mask_words            77895 non-null  object 
 8   number_of_potenial_mask_words  77895 non-null  int64  
 9   potenial_mask_words_BY_words   77895 non-null  float64
dtypes: float64(1), int64(5), object(4)
memory usage: 6.5+ MB


#### Data Cleaning as done in https://www.kaggle.com/code/datafan07/disaster-tweets-nlp-eda-bert-with-transformers 

In [ ]:
# Core packages for text processing.

import string
import re

# Libraries for text preprocessing.

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Some basic helper functions to clean text by removing urls, emojis, html tags and punctuations.

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

# Applying helper functions

sample_df['text_clean'] = sample_df['consumer_complaint_narrative'].apply(lambda x: remove_URL(x))
sample_df['text_clean'] = sample_df['text_clean'].apply(lambda x: remove_emoji(x))
sample_df['text_clean'] = sample_df['text_clean'].apply(lambda x: remove_html(x))
sample_df['text_clean'] = sample_df['text_clean'].apply(lambda x: remove_punct(x))

In [ ]:
sample_df['text_clean'].head()

1039    My account  XXXX at Bofa was charged a NSF fee...
264     XXXX2016 I traveled from XXXX  where we live  ...
1515    I had my mortgage on auto pay from my bank I h...
1400    XXXX2016 on of our newly opened business accou...
1407    I issued a Check for 710000 against my account...
Name: text_clean, dtype: object

In [ ]:
# Tokenizing 'text_clean'
sample_df['tokenized'] = sample_df['text_clean'].apply(word_tokenize)
sample_df.head()

,consumer_complaint_narrative,product,split_words_whitespaces,number_of_words,number_of_charachters,charachters_by_words,number_of_unique_words,potenial_mask_words,number_of_potenial_mask_words,potenial_mask_words_BY_words,text_clean,tokenized
1039,My account # XXXX at Bofa was charged a NSF fe...,Bank account or service,"['My', 'account', '#', 'XXXX', 'at', 'Bofa', '...",85,457,5,60,"['XXXX', 'XXXX/XXXX/2016']",2,0.023529,My account XXXX at Bofa was charged a NSF fee...,"[My, account, XXXX, at, Bofa, was, charged, a,..."
264,XX/XX/2016 I traveled from XXXX ( where we liv...,Bank account or service,"['XX/XX/2016', 'I', 'traveled', 'from', 'XXXX'...",352,1892,5,183,"['XX/XX/2016', 'XXXX', 'XXXX,', 'XXXX.', 'XXXX...",29,0.082386,XXXX2016 I traveled from XXXX where we live ...,"[XXXX2016, I, traveled, from, XXXX, where, we,..."
1515,I had my mortgage on auto pay from my bank. I ...,Bank account or service,"['I', 'had', 'my', 'mortgage', 'on', 'auto', '...",50,272,5,40,['XXXX'],1,0.020000,I had my mortgage on auto pay from my bank I h...,"[I, had, my, mortgage, on, auto, pay, from, my..."
1400,XX/XX/2016 on of our newly opened business acc...,Bank account or service,"['XX/XX/2016', 'on', 'of', 'our', 'newly', 'op...",182,1018,5,121,['XX/XX/2016'],1,0.005495,XXXX2016 on of our newly opened business accou...,"[XXXX2016, on, of, our, newly, opened, busines..."
1407,I issued a Check for {$7100.00} against my acc...,Bank account or service,"['I', 'issued', 'a', 'Check', 'for', '{$7100.0...",150,843,5,95,[],0,0.000000,I issued a Check for 710000 against my account...,"[I, issued, a, Check, for, 710000, against, my..."


In [ ]:
# Lower casing clean text.

sample_df['lower'] = sample_df['tokenized'].apply(
    lambda x: [word.lower() for word in x])
sample_df.head()

,consumer_complaint_narrative,product,split_words_whitespaces,number_of_words,number_of_charachters,charachters_by_words,number_of_unique_words,potenial_mask_words,number_of_potenial_mask_words,potenial_mask_words_BY_words,text_clean,tokenized,lower
1039,My account # XXXX at Bofa was charged a NSF fe...,Bank account or service,"['My', 'account', '#', 'XXXX', 'at', 'Bofa', '...",85,457,5,60,"['XXXX', 'XXXX/XXXX/2016']",2,0.023529,My account XXXX at Bofa was charged a NSF fee...,"[My, account, XXXX, at, Bofa, was, charged, a,...","[my, account, xxxx, at, bofa, was, charged, a,..."
264,XX/XX/2016 I traveled from XXXX ( where we liv...,Bank account or service,"['XX/XX/2016', 'I', 'traveled', 'from', 'XXXX'...",352,1892,5,183,"['XX/XX/2016', 'XXXX', 'XXXX,', 'XXXX.', 'XXXX...",29,0.082386,XXXX2016 I traveled from XXXX where we live ...,"[XXXX2016, I, traveled, from, XXXX, where, we,...","[xxxx2016, i, traveled, from, xxxx, where, we,..."
1515,I had my mortgage on auto pay from my bank. I ...,Bank account or service,"['I', 'had', 'my', 'mortgage', 'on', 'auto', '...",50,272,5,40,['XXXX'],1,0.020000,I had my mortgage on auto pay from my bank I h...,"[I, had, my, mortgage, on, auto, pay, from, my...","[i, had, my, mortgage, on, auto, pay, from, my..."
1400,XX/XX/2016 on of our newly opened business acc...,Bank account or service,"['XX/XX/2016', 'on', 'of', 'our', 'newly', 'op...",182,1018,5,121,['XX/XX/2016'],1,0.005495,XXXX2016 on of our newly opened business accou...,"[XXXX2016, on, of, our, newly, opened, busines...","[xxxx2016, on, of, our, newly, opened, busines..."
1407,I issued a Check for {$7100.00} against my acc...,Bank account or service,"['I', 'issued', 'a', 'Check', 'for', '{$7100.0...",150,843,5,95,[],0,0.000000,I issued a Check for 710000 against my account...,"[I, issued, a, Check, for, 710000, against, my...","[i, issued, a, check, for, 710000, against, my..."


In [ ]:
# Removing stopwords.
stop = set(stopwords.words('english'))
sample_df['stopwords_removed'] = sample_df['lower'].apply(
    lambda x: [word for word in x if word not in stop])
sample_df.head()

,consumer_complaint_narrative,product,split_words_whitespaces,number_of_words,number_of_charachters,charachters_by_words,number_of_unique_words,potenial_mask_words,number_of_potenial_mask_words,potenial_mask_words_BY_words,text_clean,tokenized,lower,stopwords_removed
1039,My account # XXXX at Bofa was charged a NSF fe...,Bank account or service,"['My', 'account', '#', 'XXXX', 'at', 'Bofa', '...",85,457,5,60,"['XXXX', 'XXXX/XXXX/2016']",2,0.023529,My account XXXX at Bofa was charged a NSF fee...,"[My, account, XXXX, at, Bofa, was, charged, a,...","[my, account, xxxx, at, bofa, was, charged, a,...","[account, xxxx, bofa, charged, nsf, fee, xxxxx..."
264,XX/XX/2016 I traveled from XXXX ( where we liv...,Bank account or service,"['XX/XX/2016', 'I', 'traveled', 'from', 'XXXX'...",352,1892,5,183,"['XX/XX/2016', 'XXXX', 'XXXX,', 'XXXX.', 'XXXX...",29,0.082386,XXXX2016 I traveled from XXXX where we live ...,"[XXXX2016, I, traveled, from, XXXX, where, we,...","[xxxx2016, i, traveled, from, xxxx, where, we,...","[xxxx2016, traveled, xxxx, live, xxxx, xxxx, r..."
1515,I had my mortgage on auto pay from my bank. I ...,Bank account or service,"['I', 'had', 'my', 'mortgage', 'on', 'auto', '...",50,272,5,40,['XXXX'],1,0.020000,I had my mortgage on auto pay from my bank I h...,"[I, had, my, mortgage, on, auto, pay, from, my...","[i, had, my, mortgage, on, auto, pay, from, my...","[mortgage, auto, pay, bank, xxxx, accounts, ch..."
1400,XX/XX/2016 on of our newly opened business acc...,Bank account or service,"['XX/XX/2016', 'on', 'of', 'our', 'newly', 'op...",182,1018,5,121,['XX/XX/2016'],1,0.005495,XXXX2016 on of our newly opened business accou...,"[XXXX2016, on, of, our, newly, opened, busines...","[xxxx2016, on, of, our, newly, opened, busines...","[xxxx2016, newly, opened, business, account, t..."
1407,I issued a Check for {$7100.00} against my acc...,Bank account or service,"['I', 'issued', 'a', 'Check', 'for', '{$7100.0...",150,843,5,95,[],0,0.000000,I issued a Check for 710000 against my account...,"[I, issued, a, Check, for, 710000, against, my...","[i, issued, a, check, for, 710000, against, my...","[issued, check, 710000, account, deposited, an..."


In [ ]:
# Applying part of speech tags.
sample_df['pos_tags'] = sample_df['stopwords_removed'].apply(nltk.tag.pos_tag)
sample_df.head()

,consumer_complaint_narrative,product,split_words_whitespaces,number_of_words,number_of_charachters,charachters_by_words,number_of_unique_words,potenial_mask_words,number_of_potenial_mask_words,potenial_mask_words_BY_words,text_clean,tokenized,lower,stopwords_removed,pos_tags
1039,My account # XXXX at Bofa was charged a NSF fe...,Bank account or service,"['My', 'account', '#', 'XXXX', 'at', 'Bofa', '...",85,457,5,60,"['XXXX', 'XXXX/XXXX/2016']",2,0.023529,My account XXXX at Bofa was charged a NSF fee...,"[My, account, XXXX, at, Bofa, was, charged, a,...","[my, account, xxxx, at, bofa, was, charged, a,...","[account, xxxx, bofa, charged, nsf, fee, xxxxx...","[(account, NN), (xxxx, NNP), (bofa, NN), (char..."
264,XX/XX/2016 I traveled from XXXX ( where we liv...,Bank account or service,"['XX/XX/2016', 'I', 'traveled', 'from', 'XXXX'...",352,1892,5,183,"['XX/XX/2016', 'XXXX', 'XXXX,', 'XXXX.', 'XXXX...",29,0.082386,XXXX2016 I traveled from XXXX where we live ...,"[XXXX2016, I, traveled, from, XXXX, where, we,...","[xxxx2016, i, traveled, from, xxxx, where, we,...","[xxxx2016, traveled, xxxx, live, xxxx, xxxx, r...","[(xxxx2016, NN), (traveled, VBD), (xxxx, NNP),..."
1515,I had my mortgage on auto pay from my bank. I ...,Bank account or service,"['I', 'had', 'my', 'mortgage', 'on', 'auto', '...",50,272,5,40,['XXXX'],1,0.020000,I had my mortgage on auto pay from my bank I h...,"[I, had, my, mortgage, on, auto, pay, from, my...","[i, had, my, mortgage, on, auto, pay, from, my...","[mortgage, auto, pay, bank, xxxx, accounts, ch...","[(mortgage, NN), (auto, NN), (pay, VBP), (bank..."
1400,XX/XX/2016 on of our newly opened business acc...,Bank account or service,"['XX/XX/2016', 'on', 'of', 'our', 'newly', 'op...",182,1018,5,121,['XX/XX/2016'],1,0.005495,XXXX2016 on of our newly opened business accou...,"[XXXX2016, on, of, our, newly, opened, busines...","[xxxx2016, on, of, our, newly, opened, busines...","[xxxx2016, newly, opened, business, account, t...","[(xxxx2016, JJ), (newly, RB), (opened, VBN), (..."
1407,I issued a Check for {$7100.00} against my acc...,Bank account or service,"['I', 'issued', 'a', 'Check', 'for', '{$7100.0...",150,843,5,95,[],0,0.000000,I issued a Check for 710000 against my account...,"[I, issued, a, Check, for, 710000, against, my...","[i, issued, a, check, for, 710000, against, my...","[issued, check, 710000, account, deposited, an...","[(issued, VBN), (check, VB), (710000, CD), (ac..."


In [ ]:
# Converting part of speeches to wordnet format.

def get_wordnet_pos(tag):
  if tag.startswith('J'):
    return wordnet.ADJ
  elif tag.startswith('V'):
    return wordnet.VERB
  elif tag.startswith('N'):
    return wordnet.NOUN
  elif tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

sample_df['wordnet_pos'] = sample_df['pos_tags'].apply(
    lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

sample_df.head()

,consumer_complaint_narrative,product,split_words_whitespaces,number_of_words,number_of_charachters,charachters_by_words,number_of_unique_words,potenial_mask_words,number_of_potenial_mask_words,potenial_mask_words_BY_words,text_clean,tokenized,lower,stopwords_removed,pos_tags,wordnet_pos
1039,My account # XXXX at Bofa was charged a NSF fe...,Bank account or service,"['My', 'account', '#', 'XXXX', 'at', 'Bofa', '...",85,457,5,60,"['XXXX', 'XXXX/XXXX/2016']",2,0.023529,My account XXXX at Bofa was charged a NSF fee...,"[My, account, XXXX, at, Bofa, was, charged, a,...","[my, account, xxxx, at, bofa, was, charged, a,...","[account, xxxx, bofa, charged, nsf, fee, xxxxx...","[(account, NN), (xxxx, NNP), (bofa, NN), (char...","[(account, n), (xxxx, n), (bofa, n), (charged,..."
264,XX/XX/2016 I traveled from XXXX ( where we liv...,Bank account or service,"['XX/XX/2016', 'I', 'traveled', 'from', 'XXXX'...",352,1892,5,183,"['XX/XX/2016', 'XXXX', 'XXXX,', 'XXXX.', 'XXXX...",29,0.082386,XXXX2016 I traveled from XXXX where we live ...,"[XXXX2016, I, traveled, from, XXXX, where, we,...","[xxxx2016, i, traveled, from, xxxx, where, we,...","[xxxx2016, traveled, xxxx, live, xxxx, xxxx, r...","[(xxxx2016, NN), (traveled, VBD), (xxxx, NNP),...","[(xxxx2016, n), (traveled, v), (xxxx, n), (liv..."
1515,I had my mortgage on auto pay from my bank. I ...,Bank account or service,"['I', 'had', 'my', 'mortgage', 'on', 'auto', '...",50,272,5,40,['XXXX'],1,0.020000,I had my mortgage on auto pay from my bank I h...,"[I, had, my, mortgage, on, auto, pay, from, my...","[i, had, my, mortgage, on, auto, pay, from, my...","[mortgage, auto, pay, bank, xxxx, accounts, ch...","[(mortgage, NN), (auto, NN), (pay, VBP), (bank...","[(mortgage, n), (auto, n), (pay, v), (bank, n)..."
1400,XX/XX/2016 on of our newly opened business acc...,Bank account or service,"['XX/XX/2016', 'on', 'of', 'our', 'newly', 'op...",182,1018,5,121,['XX/XX/2016'],1,0.005495,XXXX2016 on of our newly opened business accou...,"[XXXX2016, on, of, our, newly, opened, busines...","[xxxx2016, on, of, our, newly, opened, busines...","[xxxx2016, newly, opened, business, account, t...","[(xxxx2016, JJ), (newly, RB), (opened, VBN), (...","[(xxxx2016, a), (newly, r), (opened, v), (busi..."
1407,I issued a Check for {$7100.00} against my acc...,Bank account or service,"['I', 'issued', 'a', 'Check', 'for', '{$7100.0...",150,843,5,95,[],0,0.000000,I issued a Check for 710000 against my account...,"[I, issued, a, Check, for, 710000, against, my...","[i, issued, a, check, for, 710000, against, my...","[issued, check, 710000, account, deposited, an...","[(issued, VBN), (check, VB), (710000, CD), (ac...","[(issued, v), (check, v), (710000, n), (accoun..."


In [ ]:
# Applying word lemmatizer.

wnl = WordNetLemmatizer()
sample_df['lemmatized'] = sample_df['wordnet_pos'].apply(
    lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
sample_df['lemmatized'] = sample_df['lemmatized'].apply(
    lambda x: [word for word in x if word not in stop])
sample_df['lemma_str'] = [' '.join(map(str, l)) for l in sample_df['lemmatized']]
sample_df.head()

,consumer_complaint_narrative,product,split_words_whitespaces,number_of_words,number_of_charachters,charachters_by_words,number_of_unique_words,potenial_mask_words,number_of_potenial_mask_words,potenial_mask_words_BY_words,text_clean,tokenized,lower,stopwords_removed,pos_tags,wordnet_pos,lemmatized,lemma_str
1039,My account # XXXX at Bofa was charged a NSF fe...,Bank account or service,"['My', 'account', '#', 'XXXX', 'at', 'Bofa', '...",85,457,5,60,"['XXXX', 'XXXX/XXXX/2016']",2,0.023529,My account XXXX at Bofa was charged a NSF fee...,"[My, account, XXXX, at, Bofa, was, charged, a,...","[my, account, xxxx, at, bofa, was, charged, a,...","[account, xxxx, bofa, charged, nsf, fee, xxxxx...","[(account, NN), (xxxx, NNP), (bofa, NN), (char...","[(account, n), (xxxx, n), (bofa, n), (charged,...","[account, xxxx, bofa, charge, nsf, fee, xxxxxx...",account xxxx bofa charge nsf fee xxxxxxxx2016 ...
264,XX/XX/2016 I traveled from XXXX ( where we liv...,Bank account or service,"['XX/XX/2016', 'I', 'traveled', 'from', 'XXXX'...",352,1892,5,183,"['XX/XX/2016', 'XXXX', 'XXXX,', 'XXXX.', 'XXXX...",29,0.082386,XXXX2016 I traveled from XXXX where we live ...,"[XXXX2016, I, traveled, from, XXXX, where, we,...","[xxxx2016, i, traveled, from, xxxx, where, we,...","[xxxx2016, traveled, xxxx, live, xxxx, xxxx, r...","[(xxxx2016, NN), (traveled, VBD), (xxxx, NNP),...","[(xxxx2016, n), (traveled, v), (xxxx, n), (liv...","[xxxx2016, travel, xxxx, live, xxxx, xxxx, rem...",xxxx2016 travel xxxx live xxxx xxxx remain xxx...
1515,I had my mortgage on auto pay from my bank. I ...,Bank account or service,"['I', 'had', 'my', 'mortgage', 'on', 'auto', '...",50,272,5,40,['XXXX'],1,0.020000,I had my mortgage on auto pay from my bank I h...,"[I, had, my, mortgage, on, auto, pay, from, my...","[i, had, my, mortgage, on, auto, pay, from, my...","[mortgage, auto, pay, bank, xxxx, accounts, ch...","[(mortgage, NN), (auto, NN), (pay, VBP), (bank...","[(mortgage, n), (auto, n), (pay, v), (bank, n)...","[mortgage, auto, pay, bank, xxxx, account, che...",mortgage auto pay bank xxxx account check acco...
1400,XX/XX/2016 on of our newly opened business acc...,Bank account or service,"['XX/XX/2016', 'on', 'of', 'our', 'newly', 'op...",182,1018,5,121,['XX/XX/2016'],1,0.005495,XXXX2016 on of our newly opened business accou...,"[XXXX2016, on, of, our, newly, opened, busines...","[xxxx2016, on, of, our, newly, opened, busines...","[xxxx2016, newly, opened, business, account, t...","[(xxxx2016, JJ), (newly, RB), (opened, VBN), (...","[(xxxx2016, a), (newly, r), (opened, v), (busi...","[xxxx2016, newly, open, business, account, tak...",xxxx2016 newly open business account take 1000...
1407,I issued a Check for {$7100.00} against my acc...,Bank account or service,"['I', 'issued', 'a', 'Check', 'for', '{$7100.0...",150,843,5,95,[],0,0.000000,I issued a Check for 710000 against my account...,"[I, issued, a, Check, for, 710000, against, my...","[i, issued, a, check, for, 710000, against, my...","[issued, check, 710000, account, deposited, an...","[(issued, VBN), (check, VB), (710000, CD), (ac...","[(issued, v), (check, v), (710000, n), (accoun...","[issue, check, 710000, account, deposit, anoth...",issue check 710000 account deposit another ban...


In [ ]:
sample_df

,consumer_complaint_narrative,product,split_words_whitespaces,number_of_words,number_of_charachters,charachters_by_words,number_of_unique_words,potenial_mask_words,number_of_potenial_mask_words,potenial_mask_words_BY_words,text_clean,tokenized,lower,stopwords_removed,pos_tags,wordnet_pos,lemmatized,lemma_str
1039,My account # XXXX at Bofa was charged a NSF fe...,Bank account or service,"['My', 'account', '#', 'XXXX', 'at', 'Bofa', '...",85,457,5,60,"['XXXX', 'XXXX/XXXX/2016']",2,0.023529,My account XXXX at Bofa was charged a NSF fee...,"[My, account, XXXX, at, Bofa, was, charged, a,...","[my, account, xxxx, at, bofa, was, charged, a,...","[account, xxxx, bofa, charged, nsf, fee, xxxxx...","[(account, NN), (xxxx, NNP), (bofa, NN), (char...","[(account, n), (xxxx, n), (bofa, n), (charged,...","[account, xxxx, bofa, charge, nsf, fee, xxxxxx...",account xxxx bofa charge nsf fee xxxxxxxx2016 ...
264,XX/XX/2016 I traveled from XXXX ( where we liv...,Bank account or service,"['XX/XX/2016', 'I', 'traveled', 'from', 'XXXX'...",352,1892,5,183,"['XX/XX/2016', 'XXXX', 'XXXX,', 'XXXX.', 'XXXX...",29,0.082386,XXXX2016 I traveled from XXXX where we live ...,"[XXXX2016, I, traveled, from, XXXX, where, we,...","[xxxx2016, i, traveled, from, xxxx, where, we,...","[xxxx2016, traveled, xxxx, live, xxxx, xxxx, r...","[(xxxx2016, NN), (traveled, VBD), (xxxx, NNP),...","[(xxxx2016, n), (traveled, v), (xxxx, n), (liv...","[xxxx2016, travel, xxxx, live, xxxx, xxxx, rem...",xxxx2016 travel xxxx live xxxx xxxx remain xxx...
1515,I had my mortgage on auto pay from my bank. I ...,Bank account or service,"['I', 'had', 'my', 'mortgage', 'on', 'auto', '...",50,272,5,40,['XXXX'],1,0.020000,I had my mortgage on auto pay from my bank I h...,"[I, had, my, mortgage, on, auto, pay, from, my...","[i, had, my, mortgage, on, auto, pay, from, my...","[mortgage, auto, pay, bank, xxxx, accounts, ch...","[(mortgage, NN), (auto, NN), (pay, VBP), (bank...","[(mortgage, n), (auto, n), (pay, v), (bank, n)...","[mortgage, auto, pay, bank, xxxx, account, che...",mortgage auto pay bank xxxx account check acco...
1400,XX/XX/2016 on of our newly opened business acc...,Bank account or service,"['XX/XX/2016', 'on', 'of', 'our', 'newly', 'op...",182,1018,5,121,['XX/XX/2016'],1,0.005495,XXXX2016 on of our newly opened business accou...,"[XXXX2016, on, of, our, newly, opened, busines...","[xxxx2016, on, of, our, newly, opened, busines...","[xxxx2016, newly, opened, business, account, t...","[(xxxx2016, JJ), (newly, RB), (opened, VBN), (...","[(xxxx2016, a), (newly, r), (opened, v), (busi...","[xxxx2016, newly, open, business, account, tak...",xxxx2016 newly open business account take 1000...
1407,I issued a Check for {$7100.00} against my acc...,Bank account or service,"['I', 'issued', 'a', 'Check', 'for', '{$7100.0...",150,843,5,95,[],0,0.000000,I issued a Check for 710000 against my account...,"[I, issued, a, Check, for, 710000, against, my...","[i, issued, a, check, for, 710000, against, my...","[issued, check, 710000, account, deposited, an...","[(issued, VBN), (check, VB), (710000, CD), (ac...","[(issued, v), (check, v), (710000, n), (accoun...","[issue, check, 710000, account, deposit, anoth...",issue check 710000 account deposit another ban...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78959,A finance officer named XXXX is repeatedly usi...,Vehicle loan or lease,"['A', 'finance', 'officer', 'named', 'XXXX', '...",54,300,5,44,"['XXXX', 'XX/XX/2018.', 'XXXX', 'XXXX']",4,0.074074,A finance officer named XXXX is repeatedly usi...,"[A, finance, officer, named, XXXX, is, repeate...","[a, finance, officer, named, xxxx, is, repeate...","[finance, officer, named, xxxx, repeatedly, us...","[(finance, NN), (officer, NN), (named, VBN), (...","[(finance, n), (officer, n), (named, v), (xxxx...","[finance, officer, name, xxxx, repeatedly, use...",finance officer name xxxx repeatedly use credi...
79104,On XX/XX/19 I refinanced my auto loan with a n...,Vehicle l

In [ ]:
sample_df_lemma_str_no_mask = sample_df.loc[:,['lemma_str','product']]
sample_df_lemma_str_no_mask.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77895 entries, 1039 to 79231
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lemma_str  77895 non-null  object
 1   product    77895 non-null  object
dtypes: object(2)
memory usage: 3.8+ MB


In [ ]:
#Define the strings to mask
mask_words_list =['XX/XX/XXXX','XX-XX-XXXX', #DATE mm/dd/yyyy mm-dd-yyyy
                  'XXXX XXXX XXXX XXXX XXXX','XXXX-XXXX-XXXX-XXXX',#CREDIT or PREPAID CARD NUMBER
                  'XXXX XXXX XXXX XXXX','XXXX XXXX XXXX','XXXX-XXXX-XXXX','XXXX-XXXX','XXXX XXXX',
                  'XXX-XX-XXXX','XXX-XXX','XX-XXXX',
                  'XXXXXXXXXXXXXXXXXX','XXXXXXXXXXXXXXXXX', 'XXXXXXXXXXXXXXXX', 'XXXXXXXXXXXXXXX', 'XXXXXXXXXXXXXX',# BANK ACCOUNT NUMBER
                  'XXXXXXXXXXXXX', 'XXXXXXXXXXXX', 'XXXXXXXXXXX',                                                   # RANGES FROM 12 TO 18 DIGITS
                  'XXXXXXXXXX','XXXXXXXXX'          #ROUTING NUMBER IS 9 DIGIT
                  'XXXX','XXX','XX']

mask_words_list_lower =[mask_word.lower() for mask_word in mask_words_list]

In [ ]:
sample_df2 =sample_df.copy()
sample_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77895 entries, 1039 to 79231
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   consumer_complaint_narrative   77895 non-null  object 
 1   product                        77895 non-null  object 
 2   split_words_whitespaces        77895 non-null  object 
 3   number_of_words                77895 non-null  int64  
 4   number_of_charachters          77895 non-null  int64  
 5   charachters_by_words           77895 non-null  int64  
 6   number_of_unique_words         77895 non-null  int64  
 7   potenial_mask_words            77895 non-null  object 
 8   number_of_potenial_mask_words  77895 non-null  int64  
 9   potenial_mask_words_BY_words   77895 non-null  float64
 10  text_clean                     77895 non-null  object 
 11  tokenized                      77895 non-null  object 
 12  lower                          77895 non-nu

In [ ]:
import spacy
# use spacy with the dependency parse 
spacy_nlp = spacy.load("en_core_web_sm")

In [ ]:
from tqdm import tqdm
# instantiate
tqdm.pandas()
sample_df2['spacy_doc']= sample_df2['lemma_str'].progress_apply(lambda x :list(spacy_nlp.pipe([x]))[0])
print("\n\nSpacy Doc Completed")

100%|██████████| 77895/77895 [26:04<00:00, 49.78it/s]



Spacy Doc Completed


In [ ]:
#working
import re
# Function to identify the tokens and named entities that have to be MASKED replace them with ' <MASK> '
def change_details(word):
  if word.like_email or word.like_url:
    return '<MASK>'
  elif any(mask_word in word.string for mask_word in mask_words_list):
    return '<MASK>'
  elif word.is_stop:
    return ''
  #elif (( word.string != '.' )|(word.string != '. ')|( word.string != ' .' )|(word.string != ' . ' )):
  elif (len(re.findall('\.',word.string)) < 1) :
    if word.is_punct:
      return ''
  return word.string


# Function where each token of spacy doc is passed through change_details()
def change_text(doc):
  # Passing each token through change_details() function.
  new_tokens = map(change_details,doc)
  new_text = str(' '.join(new_tokens))
  # replace more than one white space in the string with one white space
  new_text = re.sub(' +', ' ',new_text)
  new_text = new_text.replace(' .', '.')
  new_text = new_text.replace('\n', '')
  return new_text

In [ ]:
sample_df2['masked_lemma_str']= sample_df2['spacy_doc'].progress_apply(lambda x: change_text(x))
print("\n\nText Transformation Completed")

100%|██████████| 77895/77895 [01:27<00:00, 892.31it/s]



Text Transformation Completed


In [ ]:
#Display an example of how the Text is changed
pd.set_option('display.max_colwidth', None)
display(sample_df2.loc[:,['spacy_doc','masked_lemma_str']][:1])

,spacy_doc,masked_lemma_str
1039,"(account, xxxx, bofa, charge, nsf, fee, xxxxxxxx2016, 3500, noticed, account, activity, merchant, deposit, make, day, sure, account, charge, fee, believe, enough, fund, transit, sufficient, enough, take, care, fee, bank, charge, fee, unjustly, spite, fact, merchant, service, account, also, bofa, daily, deposit)",account <MASK> bofa charge nsf fee <MASK> 3500 noticed account activity merchant deposit day sure account charge fee believe fund transit sufficient care fee bank charge fee unjustly spite fact merchant service account bofa daily deposit


In [ ]:
#Split 'masked_lemma_str' into substrings whenever whitespace occur
sample_df2['split_words_whitespaces'] = sample_df2['masked_lemma_str'].apply(lambda x: x.split())
#Count the number of substrings in 'split_words_whitespaces'
sample_df2['number_of_words'] = sample_df2['split_words_whitespaces'].apply(lambda x: len(x))
#Count the number of charachters in  'masked_lemma_str'
sample_df2['number_of_charachters'] = sample_df2['masked_lemma_str'].apply(lambda x: len(x))
#Calculate the ratio of number of charachters by number of words
sample_df2['charachters_by_words'] = sample_df2['number_of_charachters'] // sample_df2['number_of_words']
#Count the number of unique strings in 'split_words_whitespaces'
sample_df2['number_of_unique_words'] = sample_df2['split_words_whitespaces'].apply(lambda x : len(set(x)))
#Count the number of '<MASK>' strings in 'masked_lemma_str'
sample_df2['number_of_<MASK>'] = sample_df2['masked_lemma_str'].apply(lambda x : x.count('<MASK>'))
#Count the number of '<MASK>' by 'number of words'
sample_df2['<MASK>_BY_WORDS'] = sample_df2['number_of_<MASK>']/sample_df2['number_of_words']
sample_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77895 entries, 1039 to 79231
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   consumer_complaint_narrative   77895 non-null  object 
 1   product                        77895 non-null  object 
 2   split_words_whitespaces        77895 non-null  object 
 3   number_of_words                77895 non-null  int64  
 4   number_of_charachters          77895 non-null  int64  
 5   charachters_by_words           77895 non-null  int64  
 6   number_of_unique_words         77895 non-null  int64  
 7   potenial_mask_words            77895 non-null  object 
 8   number_of_potenial_mask_words  77895 non-null  int64  
 9   potenial_mask_words_BY_words   77895 non-null  float64
 10  text_clean                     77895 non-null  object 
 11  tokenized                      77895 non-null  object 
 12  lower                          77895 non-nu

In [ ]:
sample_df3 = sample_df2[sample_df2['<MASK>_BY_WORDS'].le(0.3)]
sample_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73171 entries, 1039 to 79231
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   consumer_complaint_narrative   73171 non-null  object 
 1   product                        73171 non-null  object 
 2   split_words_whitespaces        73171 non-null  object 
 3   number_of_words                73171 non-null  int64  
 4   number_of_charachters          73171 non-null  int64  
 5   charachters_by_words           73171 non-null  int64  
 6   number_of_unique_words         73171 non-null  int64  
 7   potenial_mask_words            73171 non-null  object 
 8   number_of_potenial_mask_words  73171 non-null  int64  
 9   potenial_mask_words_BY_words   73171 non-null  float64
 10  text_clean                     73171 non-null  object 
 11  tokenized                      73171 non-null  object 
 12  lower                          73171 non-nu

In [ ]:
sample_df4 = sample_df2[sample_df2['<MASK>_BY_WORDS'].le(0.1)]
sample_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43220 entries, 1039 to 79231
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   consumer_complaint_narrative   43220 non-null  object 
 1   product                        43220 non-null  object 
 2   split_words_whitespaces        43220 non-null  object 
 3   number_of_words                43220 non-null  int64  
 4   number_of_charachters          43220 non-null  int64  
 5   charachters_by_words           43220 non-null  int64  
 6   number_of_unique_words         43220 non-null  int64  
 7   potenial_mask_words            43220 non-null  object 
 8   number_of_potenial_mask_words  43220 non-null  int64  
 9   potenial_mask_words_BY_words   43220 non-null  float64
 10  text_clean                     43220 non-null  object 
 11  tokenized                      43220 non-null  object 
 12  lower                          43220 non-nu

In [ ]:
sample_df5 = sample_df2[sample_df2['<MASK>_BY_WORDS'].le(0.2)]
sample_df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65356 entries, 1039 to 79231
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   consumer_complaint_narrative   65356 non-null  object 
 1   product                        65356 non-null  object 
 2   split_words_whitespaces        65356 non-null  object 
 3   number_of_words                65356 non-null  int64  
 4   number_of_charachters          65356 non-null  int64  
 5   charachters_by_words           65356 non-null  int64  
 6   number_of_unique_words         65356 non-null  int64  
 7   potenial_mask_words            65356 non-null  object 
 8   number_of_potenial_mask_words  65356 non-null  int64  
 9   potenial_mask_words_BY_words   65356 non-null  float64
 10  text_clean                     65356 non-null  object 
 11  tokenized                      65356 non-null  object 
 12  lower                          65356 non-nu

## Download the Sample Data

In [ ]:
sample_df_lemma_str_no_mask.to_csv("/content/drive/MyDrive/Complaints_csv/Experiment5/SAMPLE_lemma_str_no_mask.csv", encoding='utf-8', index=False)
print("\n\nDownload Completed")



Download Completed


In [ ]:
#Download the sample data
sample_df3.to_csv("/content/drive/MyDrive/Complaints_csv/Experiment5/SAMPLE_DOC_30_PER_MASK_EXP_5.csv", encoding='utf-8', index=False)
print("\n\nDownload Completed")



Download Completed


In [ ]:
#Download the sample data
sample_df4.to_csv("/content/drive/MyDrive/Complaints_csv/Experiment5/SAMPLE_DOC_10_PER_MASK_EXP_5.csv", encoding='utf-8', index=False)
print("\n\nDownload Completed")



Download Completed


In [ ]:
#Download the sample data
sample_df5.to_csv("/content/drive/MyDrive/Complaints_csv/Experiment5/SAMPLE_DOC_20_PER_MASK_EXP_5.csv", encoding='utf-8', index=False)
print("\n\nDownload Completed")



Download Completed
